In [1]:
from txt_to_dict import get_txts

topics = get_txts()

import time
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from tqdm import tqdm
from random import randint

trainset = np.load('training_data_w2v_shuffled.npy',allow_pickle=True)
trainset1 = trainset[0:int(len(trainset)*0.8)]
trainset2 = trainset[int(len(trainset)*0.8):int(len(trainset)*0.9)]
testset = trainset[int(len(trainset)*0.9):len(trainset)]

In [6]:
print(type(trainset[1][0]))

<class 'list'>


In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(20*400,64)
        self.fc2 = nn.Linear(64,64)
        self.fc3 = nn.Linear(64,64)
        self.fc4 = nn.Linear(64,13)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.softmax(x, dim=1)

In [8]:
def train(net):
    EPOCHS = 6
    BATCH_SIZE = 32
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(trainset1)-BATCH_SIZE, BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            
#           X, y = torch.from_numpy(trainset1[i:i+BATCH_SIZE]).to(device)
            X = torch.empty(BATCH_SIZE, len(trainset[0][0][0])*20)
            for j in range(BATCH_SIZE):
                txt = np.array([])
                for k in range(20):
                    word = trainset1[i+j-1][0][k]
                    txt = np.append(txt, word)
                X[j] = torch.from_numpy(txt)
            y = torch.empty(BATCH_SIZE, 13)
            for j in range(BATCH_SIZE):
                y[j] = torch.from_numpy(trainset1[i+j-1][1])
            X = X.to(device)
            y = y.to(device)
            net.zero_grad()
            
            outputs = net(X)
            loss = loss_function(outputs, y)
            loss.backward()
            optimizer.step()    # Does the update

        print(f"Epoch: {epoch}. Loss: {loss}")
    return None

In [9]:
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(testset))):
            real_class = torch.argmax(torch.from_numpy(testset[i][1])).to(device)
            try:
                txt = torch.from_numpy(np.append([],testset[i][0]))
            except:
                dum = np.array([])
                for j in range(20):
                    if (len(testset[i][0][j])==20):
                        dum = np.append(dum, testset[i][0][j])
                    else:
                        dum = np.append(dum, testset[i][0][j][0])
            txt = txt.float()
            net_out = net(txt.view(-1,len(txt)).to(device))[0]  # returns a list, 
            predicted_class = torch.argmax(net_out)

            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Accuracy: ", round(correct/total, 3))
    return None

In [10]:
def fwd_pass(X, y, train = False):
    if train:
        net.zero_grad()
    outputs = net(X)
    matches  = [torch.argmax(i)==torch.argmax(j) for i, j in zip(outputs, y)]
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)

    if train:
        loss.backward()
        optimizer.step()

    return acc, loss

In [17]:
def test_(size=32):
    i = randint(0,len(testset)-size-1)
    X = torch.empty(size, len(testset[0][0][0])*20)
    for j in range(size):
        txt = np.array([])
        for k in range(20):
            word = testset[i+j-1][0][k]
            txt = np.append(txt, word)
        X[j] = torch.from_numpy(txt)
    y = torch.empty(size, 13)
    for j in range(size):
        y[j] = torch.from_numpy(testset[i+j-1][1])
    val_acc, val_loss = fwd_pass(X.view(-1,len(testset[0][0][0])*20).to(device), y.to(device))
    return val_acc, val_loss

In [31]:
device = torch.device("cuda:0")
net = Net().to(device)
print(net)
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()


Net(
  (fc1): Linear(in_features=8000, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=13, bias=True)
)


In [32]:
MODEL_NAME = f"model-{int(time.time())}"
def train_(net):
    BATCH_SIZE = 32
    EPOCHS = 10

    with open("model_w2v.log", "a") as f:
        for epoch in range(EPOCHS):
            for i in tqdm(range(0, len(trainset1)-BATCH_SIZE, BATCH_SIZE)):
                
                X = torch.empty(BATCH_SIZE, len(trainset[0][0][0])*20)
                for j in range(BATCH_SIZE):
                    txt = np.array([])
                    for k in range(20):
                        word = trainset1[i+j-1][0][k]
                        txt = np.append(txt, word)
                    X[j] = torch.from_numpy(txt)
                y = torch.empty(BATCH_SIZE, 13)
                for j in range(BATCH_SIZE):
                    y[j] = torch.from_numpy(trainset1[i+j-1][1])

                X, y = X.to(device), y.to(device)

                acc, loss = fwd_pass(X, y, train=True)

                #print(f"Acc: {round(float(acc),2)}  Loss: {round(float(loss),4)}")
                #f.write(f"{MODEL_NAME},{round(time.time(),3)},train,{round(float(acc),2)},{round(float(loss),4)}\n")
                # just to show the above working, and then get out:
                if i % 50 == 0:
                    val_acc, val_loss = test_(size=32)
                    f.write(f"{MODEL_NAME},{round(time.time(),3)},{round(float(acc),2)},{round(float(loss), 4)},{round(float(val_acc),2)},{round(float(val_loss),4)}\n")
train_(net)

100%|██████████| 1049/1049 [00:14<00:00, 70.59it/s]


In [20]:
haber = "Tunus Savunma Bakanlığı, ANKA-S İHA alımı için TUSAŞ ile sözleşme imzaladı. Anlaşma, 3 adet ANKA-S İHA ve 3 adet yer kontrol istasyonunun teslimi ile 52 Tunus Hava Kuvvetleri personelinin eğitimini kapsıyor. İhracatın 80 milyon dolar tutarında olduğu belirtildi.İHA ve SİHA düzenlenen askeri operasyonlarda adından söz ettiren Türkiye, Katar ve Ukrayna'nın ardından Tunus'a da satış gerçekleştirecek.Türk Havacılık ve Uzay Sanayii (TUSAŞ), tarihinde ilk kez milli ve yerli tasarım insansız hava aracı ANKA-S'nin ihracatına imza attı.TUNUS'A ANKA-S İHRACATI2019 yılında başlayan ve anlaşmaya varılan görüşmeler sonucunda Tunus Hava Kuvvetleri Komutanlığı'na 3 adet ANKA-S İHA ve 3 adet Yer Kontrol Sistemi sağlanacak."